In [1]:
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
data=pd.read_csv('/content/train.csv')

In [10]:
data=data.drop(['id','keyword','location'],axis=1)

In [11]:
data.head(5)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
nlp=spacy.load('en_core_web_sm')

In [13]:
stop_words=nlp.Defaults.stop_words

In [15]:
punctuation=string.punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [18]:
def spacy_tokenizer(sentence):
    doc=nlp(sentence)
    mytokens=[ word.lemma_.lower().strip() for word in doc ]

    mytokens=[word for word in mytokens if word not in stop_words and word not in punctuation]

    return mytokens


In [19]:
sentence='you like to watch movie?'
spacy_tokenizer(sentence)

['like', 'watch', 'movie']

In [20]:
count_vector=CountVectorizer(tokenizer=spacy_tokenizer)

In [24]:
count_vector.fit_transform(['I am watching movie','i like to Play football']).toarray()

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0, 0, 1, 0, 1],
       [1, 1, 0, 1, 0]])

In [25]:
count_vector.get_feature_names_out()

array(['football', 'like', 'movie', 'play', 'watch'], dtype=object)

In [26]:
count_vector.vocabulary_

{'watch': 4, 'movie': 2, 'like': 1, 'play': 3, 'football': 0}

In [27]:
from sklearn.model_selection import train_test_split
X=data['text'] 
y=data['target']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,stratify=y)

In [29]:
X_train_vectors=count_vector.fit_transform(X_train)
X_test_vectors=count_vector.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [30]:
X_train_vectors.shape

(6090, 17205)

In [31]:
X_test_vectors.shape

(1523, 17205)

In [32]:
X_train_vectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()

In [42]:
X_train_vectors

<6090x17205 sparse matrix of type '<class 'numpy.float64'>'
	with 56054 stored elements in Compressed Sparse Row format>

In [44]:
y_train

6234    1
326     0
997     0
7269    0
2189    1
       ..
3386    1
3280    1
305     0
1648    0
7569    0
Name: target, Length: 6090, dtype: int64

In [34]:
classifier.fit(X_train_vectors,y_train)

LogisticRegression()

In [36]:
predicted = classifier.predict(X_test_vectors)
print('Accuracy:', metrics.accuracy_score(y_test, predicted))
print('Precision:', metrics.precision_score(y_test, predicted))
print('Recall:', metrics.recall_score(y_test, predicted))

Accuracy: 0.8154957321076822
Precision: 0.8209982788296041
Recall: 0.7293577981651376


In [37]:
tfidf_vector=TfidfVectorizer(tokenizer=spacy_tokenizer)
X_train_vectors=tfidf_vector.fit_transform(X_train)
X_test_vectors=tfidf_vector.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:
classifier=LogisticRegression()
classifier.fit(X_train_vectors,y_train)
predicted = classifier.predict(X_test_vectors)
print('Accuracy:', metrics.accuracy_score(y_test, predicted))
print('Precision:', metrics.precision_score(y_test, predicted))
print('Recall:', metrics.recall_score(y_test, predicted))

Accuracy: 0.8108995403808273
Precision: 0.8256227758007118
Recall: 0.709480122324159
